In [1]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

# rmsle

In [2]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values, convertExp=True):

    if convertExp:
        predicted_values = np.exp(predicted_values),
        actual_values = np.exp(actual_values)

    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)

    # 평균을 낸다.
    mean_difference = difference.mean()

    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)

    return score

## 전에만든 정제된 데이터프레임

In [3]:
df = pd.read_csv('data_filter.csv')

In [22]:
df.drop(['Unnamed: 0'], axis = 1, inplace=True)

## 인코딩

In [23]:
from sklearn.preprocessing import LabelEncoder
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

## 명목형변수 카테고리화

In [24]:
categorical_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']

for i in categorical_features:
    df_num[i] = df_num[i].astype('category')

In [25]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   CARD_SIDO_NM  1057394 non-null  category
 1   STD_CLSS_NM   1057394 non-null  category
 2   HOM_SIDO_NM   1057394 non-null  category
 3   AGE           1057394 non-null  category
 4   SEX_CTGO_CD   1057394 non-null  category
 5   FLC           1057394 non-null  category
 6   year          1057394 non-null  category
 7   month         1057394 non-null  category
 8   CSTMR_CNT     1057394 non-null  int64   
 9   AMT           1057394 non-null  int64   
 10  CNT           1057394 non-null  int64   
dtypes: category(8), int64(3)
memory usage: 32.3 MB


## AMT빼고 트레인 데이터셋 구성

In [26]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
X_train = train_num.drop([ 'AMT'], axis=1)
y_train = train_num['AMT']

## CNT, CSTMR_CNT 예측해놓은거 불러옴.

In [27]:
X_test = pd.read_csv("submission_template.csv")

In [28]:
X_test.drop("Unnamed: 0", axis = 1, inplace=True)

In [29]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   CARD_SIDO_NM  1658860 non-null  int64
 1   STD_CLSS_NM   1658860 non-null  int64
 2   HOM_SIDO_NM   1658860 non-null  int64
 3   AGE           1658860 non-null  int64
 4   SEX_CTGO_CD   1658860 non-null  int64
 5   FLC           1658860 non-null  int64
 6   year          1658860 non-null  int64
 7   month         1658860 non-null  int64
 8   CNT           1658860 non-null  int64
 9   CSTMR_CNT     1658860 non-null  int64
dtypes: int64(10)
memory usage: 126.6 MB


In [30]:
categorical_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']

for i in categorical_features:
    X_test[i] = X_test[i].astype('category')

In [31]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   CARD_SIDO_NM  1658860 non-null  category
 1   STD_CLSS_NM   1658860 non-null  category
 2   HOM_SIDO_NM   1658860 non-null  category
 3   AGE           1658860 non-null  category
 4   SEX_CTGO_CD   1658860 non-null  category
 5   FLC           1658860 non-null  category
 6   year          1658860 non-null  category
 7   month         1658860 non-null  category
 8   CNT           1658860 non-null  int64   
 9   CSTMR_CNT     1658860 non-null  int64   
dtypes: category(8), int64(2)
memory usage: 38.0 MB


# 보여주려고 만들다가 보니까 CNT랑 CSTMR_CNT랑 상관계수가 너무 높아서 하나 버려도 될듯? ㅇㅇ

In [32]:
X_test.corr()

,CNT,CSTMR_CNT
CNT,1.000000,0.949906
CSTMR_CNT,0.949906,1.000000


## randomforestregressor 모델 학습

In [33]:
from sklearn.ensemble import RandomForestRegressor

lr_reg = RandomForestRegressor(n_jobs = -1, random_state = 0)
y_train_log = np.log1p(y_train)
lr_reg.fit(X_train, y_train_log)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

## 예측

In [34]:
X_train

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,CNT
149245,2,4,2,2,1,2,2019,4,5,5
554956,8,26,11,1,1,2,2020,2,3,6
918516,14,33,14,4,2,4,2019,5,10262,15027
425414,7,18,8,2,2,1,2019,10,120,164
640949,9,39,16,5,2,5,2020,1,9,9
...,...,...,...,...,...,...,...,...,...,...
359783,6,16,9,1,1,1,2019,12,382,530
152315,2,7,11,3,2,3,2020,3,3,5
963395,15,19,9,5,1,5,2020,2,13,15
117952,1,31,4,1,1,2,2019,7,3,6


In [35]:
X_test

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CNT,CSTMR_CNT
0,0,0,0,1,1,1,2020,4,4,4
1,0,0,0,1,1,1,2020,7,3,3
2,0,0,0,1,1,2,2020,4,8,8
3,0,0,0,1,1,2,2020,7,6,5
4,0,0,0,1,1,3,2020,4,10,8
...,...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,12,12
1658856,16,30,14,0,2,4,2020,4,15,21
1658857,16,30,14,0,2,4,2020,7,15,21
1658858,16,30,14,0,2,5,2020,4,15,20


In [36]:
lr_reg.predict(X_test)

array([12.29416851, 11.750191  , 12.77668126, ..., 12.22289795,
       12.2984204 , 12.21824629])

In [37]:
pred = lr_reg.predict(X_test)

In [38]:
pred = np.expm1(pred)
pred

array([218417.56264528, 126776.77157894, 353867.71261642, ...,
       203392.52885793, 219348.23100139, 202448.60957039])

In [39]:
X_test['AMT'] = np.round(pred, 0)

In [40]:
pred

array([218417.56264528, 126776.77157894, 353867.71261642, ...,
       203392.52885793, 219348.23100139, 202448.60957039])

In [41]:
X_test

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CNT,CSTMR_CNT,AMT
0,0,0,0,1,1,1,2020,4,4,4,218418.0
1,0,0,0,1,1,1,2020,7,3,3,126777.0
2,0,0,0,1,1,2,2020,4,8,8,353868.0
3,0,0,0,1,1,2,2020,7,6,5,343694.0
4,0,0,0,1,1,3,2020,4,10,8,430647.0
...,...,...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,12,12,147195.0
1658856,16,30,14,0,2,4,2020,4,15,21,220445.0
1658857,16,30,14,0,2,4,2020,7,15,21,203393.0
1658858,16,30,14,0,2,5,2020,4,15,20,219348.0


In [43]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   CARD_SIDO_NM  1658860 non-null  category
 1   STD_CLSS_NM   1658860 non-null  category
 2   HOM_SIDO_NM   1658860 non-null  category
 3   AGE           1658860 non-null  category
 4   SEX_CTGO_CD   1658860 non-null  category
 5   FLC           1658860 non-null  category
 6   year          1658860 non-null  category
 7   month         1658860 non-null  category
 8   CNT           1658860 non-null  int64   
 9   CSTMR_CNT     1658860 non-null  int64   
 10  AMT           1658860 non-null  float64 
dtypes: category(8), float64(1), int64(2)
memory usage: 50.6 MB


In [47]:
X_test['REG_YYMM'] = X_test['year'].astype('int64')*100 + X_test['month'].astype('int64')
X_test = X_test[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM','AMT']]
X_test = X_test.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [48]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394 entries, 0 to 1393
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   REG_YYMM      1394 non-null   int64   
 1   CARD_SIDO_NM  1394 non-null   category
 2   STD_CLSS_NM   1394 non-null   category
 3   AMT           1394 non-null   float64 
dtypes: category(2), float64(1), int64(1)
memory usage: 27.0 KB


In [49]:
X_test['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(X_test['CARD_SIDO_NM'])
X_test['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(X_test['STD_CLSS_NM'])

In [50]:
X_test

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.984122e+09
1,202004,강원,골프장 운영업,1.138989e+10
2,202004,강원,과실 및 채소 소매업,3.742357e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.808010e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.117270e+08
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.532998e+09
1390,202007,충북,한식 음식점업,4.966927e+10
1391,202007,충북,호텔업,3.042336e+08
1392,202007,충북,화장품 및 방향제 소매업,2.153298e+09


In [51]:
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(X_test, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_2_3.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.984122e+09
1,202004,강원,골프장 운영업,1.138989e+10
2,202004,강원,과실 및 채소 소매업,3.742357e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.808010e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.117270e+08
